#Linear regression model from scratch

In [1]:
import numpy as np
import torch

In [2]:
inputs = np.array([[73, 67, 43],
[91, 88, 64],
[87, 134, 58],
[102, 43, 37],
[69, 96, 70]], dtype='float32')

In [3]:
targets = np.array([[56],
[81],
[119],
[22],
[103]], dtype='float32')

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [5]:
weight = torch.randn(2, 3, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

In [6]:
inputs @ weight.t() + bias

tensor([[ 156.3092,  -89.8394],
        [ 206.5217, -130.5247],
        [ 266.3050, -142.4261],
        [ 133.3835,  -64.9172],
        [ 210.2741, -147.1112]], grad_fn=<AddBackward0>)

In [7]:
def model(inputs):
    return inputs @ weight.t() + bias

In [8]:
prediction = model(inputs)
print(prediction)

tensor([[ 156.3092,  -89.8394],
        [ 206.5217, -130.5247],
        [ 266.3050, -142.4261],
        [ 133.3835,  -64.9172],
        [ 210.2741, -147.1112]], grad_fn=<AddBackward0>)


##Loss function

In [9]:
diff0 = prediction - targets
torch.sum(diff0 * diff0) / diff0.numel()

tensor(27589.6094, grad_fn=<DivBackward0>)

In [10]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [11]:
loss = mse(prediction, targets)
print(loss)

tensor(27589.6094, grad_fn=<DivBackward0>)


##Compute gradients

In [12]:
loss.backward()
print(weight.grad)
print(bias.grad)

tensor([[ 10064.7227,  10518.6602,   6504.1504],
        [-15752.4639, -18232.9258, -11139.0215]])
tensor([ 118.3587, -191.1637])


##Train the model using gradient descent

In [13]:
with torch.no_grad():
    weight -= weight.grad * 1e-5
    bias -= bias.grad * 1e-5
    weight.grad.zero_()
    bias.grad.zero_()

In [14]:
prediction = model(inputs)
loss = mse(prediction, targets)
print(loss)

tensor(18858.0195, grad_fn=<DivBackward0>)


In [15]:
for i in range(100):
    prediction = model(inputs)
    loss = mse(prediction, targets)
    loss.backward()
    with torch.no_grad():
        weight -= weight.grad * 1e-5
        bias -= bias.grad * 1e-5
        weight.grad.zero_()
        bias.grad.zero_()

In [16]:
prediction = model(inputs)
loss = mse(prediction, targets)
print(loss)

tensor(319.0589, grad_fn=<DivBackward0>)


In [17]:
prediction

tensor([[ 59.1611,  64.2505],
        [ 79.8828,  76.0928],
        [120.7754, 121.2045],
        [ 30.6997,  62.1179],
        [ 92.5913,  67.1432]], grad_fn=<AddBackward0>)

In [18]:
targets

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])

In [19]:
import numpy as np
import torch

In [20]:
# Targets (apples, oranges)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [21]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [22]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [23]:
from torch.utils.data import TensorDataset

In [24]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [25]:
from torch.utils.data import DataLoader

In [26]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [27]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.]])
tensor([[119., 133.],
        [ 22.,  37.],
        [ 81., 101.],
        [103., 119.],
        [ 56.,  70.]])


In [28]:
import torch.nn as nn

In [29]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4500,  0.2528, -0.2160],
        [-0.1334, -0.0199,  0.5040]], requires_grad=True)
Parameter containing:
tensor([0.0128, 0.0159], requires_grad=True)


In [30]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4500,  0.2528, -0.2160],
         [-0.1334, -0.0199,  0.5040]], requires_grad=True),
 Parameter containing:
 tensor([0.0128, 0.0159], requires_grad=True)]

In [31]:
preds = model(inputs)
preds

tensor([[-25.1873,  10.6187],
        [-32.5147,  18.3840],
        [-17.7876,  14.9795],
        [-43.0094,   4.2041],
        [-21.8881,  24.1829]], grad_fn=<AddmmBackward0>)

In [32]:
import torch.nn.functional as F

In [33]:
loss_fn = F.mse_loss

In [34]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(9235.7285, grad_fn=<MseLossBackward0>)


In [35]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [36]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [37]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 460.5511
Epoch [20/100], Loss: 184.9180
Epoch [30/100], Loss: 160.2918
Epoch [40/100], Loss: 142.7866
Epoch [50/100], Loss: 127.4212
Epoch [60/100], Loss: 113.8613
Epoch [70/100], Loss: 101.8902
Epoch [80/100], Loss: 91.3189
Epoch [90/100], Loss: 81.9810
Epoch [100/100], Loss: 73.7296


In [38]:
prediction = model(inputs)
prediction

tensor([[ 59.3672,  73.2789],
        [ 79.8558, 102.1857],
        [120.5002, 124.7371],
        [ 33.4764,  54.2092],
        [ 90.6004, 111.7443]], grad_fn=<AddmmBackward0>)

In [39]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [40]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[55.7289, 71.6876]], grad_fn=<AddmmBackward0>)